## Group feature extraction

In [1]:
import movekit as mkit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "./datasets/fish-5-cleaned.csv"
data = mkit.read_data(path)
data = mkit.extract_features(data)
data.head()

Extracting all absolute features: 100%|██████████| 100.0/100 [00:01<00:00, 54.14it/s] 


,time,animal_id,x,y,distance,direction,turning,average_speed,average_acceleration,stopped
0,1,312,405.29,417.76,0.0,"(0.0, 0.0)",0.0,0.115306,0.005298,1
1,1,511,369.99,428.78,0.0,"(0.0, 0.0)",0.0,0.012708,0.002823,1
2,1,607,390.33,405.89,0.0,"(0.0, 0.0)",0.0,0.045118,0.003392,1
3,1,811,445.15,411.94,0.0,"(0.0, 0.0)",0.0,0.232790,0.026729,1
4,1,905,366.06,451.76,0.0,"(0.0, 0.0)",0.0,0.067000,0.001639,1


### Detecting outliers
Function performs detection of outliers, based on the KNN algorithm: user can define the regarding features for the detection, the number of the nearest neighbors taken into account for the outlier classification, the metric to calculate the distance, the method to aggregate the different distances, and the share of outliers.

In [3]:
# Detect outliers based on KNN.
# mkit.outlier_detection(dataset, features=["distance", "average_speed", "average_acceleration",
# "stopped", "turning"], contamination=0.01, n_neighbors=5, method="mean", metric="minkowski")
outs = mkit.outlier_detection(data)
# printing all rows where outliers are present
outs[outs.loc[:,"outlier"] == 1].head()

,time,animal_id,outlier,x,y,distance,direction,turning,average_speed,average_acceleration,stopped
8,2,811,1,445.48,412.26,0.459674,"(0.33, 0.32)",0.000000,0.282267,0.028174,1
2603,521,811,1,71.65,333.29,0.472652,"(0.05, 0.47)",0.437319,0.347270,0.024087,1
2608,522,811,1,71.56,334.19,0.904489,"(-0.09, 0.9)",0.978928,0.386225,0.011013,1
3837,768,607,1,387.13,57.34,5.327213,"(5.24, -0.96)",0.999938,4.605855,-0.193145,0
3869,774,905,1,425.99,62.25,0.524976,"(0.4, -0.34)",0.999999,1.379776,0.198372,0


In [4]:
# same function, different parameters
other_outs = mkit.outlier_detection(dataset = data, features = ["average_speed", "average_acceleration"], contamination = 0.05, n_neighbors = 8, method = "median", metric = "euclidean")

# printing all rows where outliers are present
other_outs[other_outs.loc[:,"outlier"] == 1].head()

,time,animal_id,outlier,x,y,distance,direction,turning,average_speed,average_acceleration,stopped
1522,305,607,1,179.53,417.02,3.268103,"(-3.26, -0.23)",0.999998,3.195286,-0.068100,0
1602,321,607,1,132.58,411.00,2.211990,"(-2.2, -0.23)",0.999645,2.088872,-0.217909,0
1607,322,607,1,130.81,410.81,1.780169,"(-1.77, -0.19)",0.999996,1.824672,-0.218531,0
1612,323,607,1,129.26,410.63,1.560417,"(-1.55, -0.18)",0.999962,1.561242,-0.214846,0
1617,324,607,1,127.95,410.58,1.310954,"(-1.31, -0.05)",0.997001,1.296308,-0.205183,0


### Group-level Analysis

Below we perform Analysis on Group-Level. This consists of:
- Group-Level averages,
- Centroid Medoid computation
- A dynamic time warping matrix, 
- A clustering over time based on absolute features,
- The centroid direction,
- The heading difference of each animal with respect to the current centroid
- The group - polarization for each timestep. 

#### Obtain group-level records for each point in time
Records consist of total group-distance covered, mean speed, mean acceleration and mean distance from centroid for each timestamp. If input doesn't contain centroid or feature data, it is calculated, showing a warning.

In [5]:
group_data = mkit.group_movement(data)
group_data.head()

Calculating centroid distances: 100%|██████████| 1000/1000 [00:04<00:00, 219.04it/s]


,total_dist,mean_speed,mean_acceleration,mean_distance_centroid
time,,,,
1,0.000000,0.094584,0.007976,29.4616
2,1.174908,0.108927,0.007736,29.5850
3,1.025155,0.122863,0.007782,29.6914
4,0.918960,0.138007,0.007898,29.7782
5,0.830461,0.155004,0.008365,29.8518


#### Obtain centroid, medoid and distance to centroid for each movement record

In [6]:
movement = mkit.centroid_medoid_computation(data, object_output = False)
movement.head()

Calculating centroid distances: 100%|██████████| 1000/1000 [00:05<00:00, 191.41it/s]


,time,animal_id,outlier,x,y,distance,direction,turning,average_speed,average_acceleration,stopped,x_centroid,y_centroid,medoid,distance_to_centroid
0,1,312,0,405.29,417.76,0.0,"(0.0, 0.0)",0.0,0.115306,0.005298,1,395.364,423.226,312,11.331
1,1,511,0,369.99,428.78,0.0,"(0.0, 0.0)",0.0,0.012708,0.002823,1,395.364,423.226,312,25.975
2,1,607,0,390.33,405.89,0.0,"(0.0, 0.0)",0.0,0.045118,0.003392,1,395.364,423.226,312,18.052
3,1,811,0,445.15,411.94,0.0,"(0.0, 0.0)",0.0,0.232790,0.026729,1,395.364,423.226,312,51.049
4,1,905,0,366.06,451.76,0.0,"(0.0, 0.0)",0.0,0.067000,0.001639,1,395.364,423.226,312,40.901


#### Get the heading difference between centroids and animal's direction
Heading difference is computed with the cosine similarity of the two direction vectors, thus ranges from -1 to 1. While 1 indicates the animal and the centroid have the same direction, -1 indicates they move in different directions.

In [7]:
centroid_dir = mkit.compute_centroid_direction(data).sort_values(['time','animal_id'])
heading_diff = mkit.get_heading_difference(data)
heading_diff.head()


Calculating centroid distances: 100%|██████████| 1000/1000 [00:02<00:00, 490.68it/s]
Computing centroid direction: 100%|██████████| 100.0/100 [00:00<00:00, 738.58it/s]
Calculating centroid distances: 100%|██████████| 1000/1000 [00:05<00:00, 185.77it/s]
Calculating heading difference: 100%|██████████| 100.0/100 [00:01<00:00, 75.39it/s]


,time,animal_id,outlier,x,y,distance,direction,turning,average_speed,average_acceleration,stopped,x_centroid,y_centroid,medoid,distance_to_centroid,centroid_direction,heading_difference
0,1,312,0,405.29,417.76,0.0,"(0.0, 0.0)",0.0,0.115306,0.005298,1,395.364,423.226,312,11.331,"(0.0, 0.0)",0.0
1,1,511,0,369.99,428.78,0.0,"(0.0, 0.0)",0.0,0.012708,0.002823,1,395.364,423.226,312,25.975,"(0.0, 0.0)",0.0
2,1,607,0,390.33,405.89,0.0,"(0.0, 0.0)",0.0,0.045118,0.003392,1,395.364,423.226,312,18.052,"(0.0, 0.0)",0.0
3,1,811,0,445.15,411.94,0.0,"(0.0, 0.0)",0.0,0.232790,0.026729,1,395.364,423.226,312,51.049,"(0.0, 0.0)",0.0
4,1,905,0,366.06,451.76,0.0,"(0.0, 0.0)",0.0,0.067000,0.001639,1,395.364,423.226,312,40.901,"(0.0, 0.0)",0.0


#### Obtain a matrix, based on dynamic time warping
Each Animal-ID is displayed in the indices, the entries reflect the similarity of the animal's trajectories based on the DTW algorithm.

In [8]:
#Obtain dynamic time warping amongst all trajectories from the animals. The lower the value for two animals is, the more similar their trajectories are based on the DTW algorithm.
#mkit.dtw_matrix(preprocessed_data, path=False, distance=euclidean)
#preprocessed_data: DataFrame containing the movement data.
#path: Boolean to specify if matrix of dtw-path gets returned as well. (the warping path for all the sequence pairs which are examined)
#distance: Specify with distance measure to use. Default: "euclidean". Other example alternatives are pdist or minkowski. (all distances defined by fastdtw package are possible.

mkit.dtw_matrix(data)

Calculating dynamic time warping: 100%|██████████| 5/5 [00:07<00:00,  1.45s/it]


,312,511,607,811,905
312,0.000000,30843.085403,32859.600139,42461.524553,37916.447829
511,30843.085403,0.000000,26931.014323,47116.708116,20967.960073
607,32859.600139,26931.014323,0.000000,39859.787924,35711.718898
811,42461.524553,47116.708116,39859.787924,0.000000,38379.806433
905,37916.447829,20967.960073,35711.718898,38379.806433,0.000000
